In [1]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client, progress, metrics
# wait for jobs to arrive, depending on the queue, this may take some time
import dask.array as da
import dask.bag as db
import numpy as np
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import os
import pyart
import tarfile
import tempfile
import shutil
from netCDF4 import num2date
import json
from time import strftime, sleep

%matplotlib inline


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [20]:
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI5.00/sgpxsaprsecI5.00.201807* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI5.00/sgpxsaprsecI5.00.201808* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI6.00/sgpxsaprsecI6.00.201807* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI6.00/sgpxsaprsecI6.00.201808* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI4.00/sgpxsaprsecI4.00.201807* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/
!adc_xfer -a /data/datastream/sgp/sgpxsaprsecI4.00/sgpxsaprsecI4.00.201808* /lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/unformatted/


Globus Transfer                        [######### ] 349/353 files (ETA 00:00:02)

In [21]:
experiment_dir = '/lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/'
formatted_subdir = 'formatted'
unformatted_subdir = 'unformatted'

unformatted_dir = os.path.join(experiment_dir, unformatted_subdir)
formatted_dir = os.path.join(experiment_dir, formatted_subdir)

all_files = os.listdir(unformatted_dir)
all_fqdn = [os.path.join(unformatted_dir, this_file) for this_file in all_files]


In [22]:
def manage_tarfile(path_and_file, 
                   experiment_location='/lustre/or-hydra/cades-arm/proj-shared/scanning_experiment/formatted'):
    def examine(fh_like):
        radar = pyart.io.read(fh_like)
        time_start = num2date(radar.time['data'][0], radar.time['units'])
        time_end = num2date(radar.time['data'][-1], radar.time['units'])
        stype = radar.scan_type
        nsweeps = radar.nsweeps
        tgates = float(radar.ngates*radar.nrays)
        zdat = radar.fields['reflectivity']['data']
        z0 = float(len(np.where(zdat > 0.)[0]))/tgates
        z10 = float(len(np.where(zdat > 10.)[0]))/tgates
        z40 = float(len(np.where(zdat > 40.)[0]))/tgates
        rdict = {'time_start' : time_start,
                'time_end' : time_end,
                 'scan_type' : stype,
                 'nsweeps' : nsweeps,
                 'z0' : z0,
                 'z10' : z10,
                 'z40' : z40,
                'expr' : radar.metadata['sigmet_task_name'].lower().strip().decode("utf-8")}

        return rdict

    def site_from_name(name):
        fullname = name.split('.')[0]
        site = fullname[-2::]
        return site

    def file_formatter(stime, site, scanmode, base, expr):
        #base/year/monthday
        
        mday = stime.strftime('%m%d')
        odir = os.path.join(base,
                            expr.lower(),
                            scanmode,
                            stime.strftime('%Y'),
                            mday)
        fname1 = 'sgpxsapr' + scanmode + site + stime.strftime('.%Y%m%d.%H%M%S')
        return odir, fname1

    top_level = os.path.split(experiment_location)[0]
    tarobj = tarfile.open(path_and_file)
    site = site_from_name(path_and_file)
    members = tarobj.getmembers()
    status = []
    for member in members:
        try:
            radar_info = examine(tarobj.extractfile(member))
            odir_radars, file_name_begin = file_formatter(radar_info['time_start'], 
                                                   site, 
                                                   radar_info['scan_type'],
                                                   experiment_location,
                                                   radar_info['expr'])

            odir_json, file_name_begin = file_formatter(radar_info['time_start'], 
                                                   site, 
                                                   radar_info['scan_type'],
                                                   os.path.join(top_level, 'summary'),
                                                   radar_info['expr'])

            try:
                if not os.path.exists(odir_radars):
                    os.makedirs(odir_radars)

                if not os.path.exists(odir_json):
                    os.makedirs(odir_json)
            except: #just wait and try again..
                sleep(1)
                if not os.path.exists(odir_radars):
                    os.makedirs(odir_radars)

                if not os.path.exists(odir_json):
                    os.makedirs(odir_json)

            fullpath = os.path.join(odir_radars, file_name_begin+'.iris')

            json_dict = {}
            strconv_keys = ['z0', 'z10', 'z40', 'nsweeps']
            for key in strconv_keys:
                json_dict.update({key : str(radar_info[key])})

            json_dict.update({'start_time' : radar_info['time_start'].strftime('%Y%m%d-%H:%M:%S'),
                             'end_time' : radar_info['time_end'].strftime('%Y%m%d-%H:%M:%S')})

            json_dict.update({'original_name' : member.name,
                             'full_path' : fullpath})

            r = json.dumps(json_dict)
            loaded_r = json.loads(r)
            with open(os.path.join(odir_json, file_name_begin+'.json'), 'w') as outfile:
                json.dump(json_dict, outfile)

            #The actuall writing
            fh = tarobj.extractfile(member)

            shutil.copyfileobj(fh, open(fullpath, 'wb'))
            fh.close()
            status.append(member.name+':OK')
        except IndexError:
            status.append(member.name+':NotOK')

        
    return status
 

In [23]:
#flist = manage_tarfile(all_fqdn[0])

In [24]:
#cluster = PBSCluster(name='dask-worker', memory='270GB', cores=36, processes=6, interface='ib0', queue='high_mem', project='arm',
#                    walltime='00:30:00')#, job-extra=['-W group_list=cades-arm'])
cluster = PBSCluster(processes = 18)
cluster.scale(4)         # Ask for ten workers
client = Client(cluster)  # Connect this local process to remote workers


```jobqueue:
  pbs:
    name: dask-worker
    cores: 36
    memory: 270GB
    processes: 6
    interface: ib0
    local-directory: $localscratch
    queue: high_mem # Can also select batch or gpu_ssd
    project: arm
    walltime: 00:30:00 #Adjust this to job size
    job-extra: ['-W group_list=cades-arm']
```

In [28]:
cluster

In [32]:
client

Client Scheduler: tcp://10.23.216.81:42873 Dashboard: http://10.23.216.81:50488/status,Cluster Workers: 72 Cores: 144 Memory: 1.08 TB


In [30]:
future = client.map(manage_tarfile, all_fqdn)



In [31]:
progress(future)


VBox()

/home/scollis/anaconda3/envs/dask/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FloatProgress).__init__(height='10px').
object.__init__() takes no parameters
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)
/home/scollis/anaconda3/envs/dask/lib/python3.6/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(HTML).__init__(width='140px').
object.__init__() takes no parameters
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


In [33]:
my_data = client.gather(future)
flat_list = [item for sublist in my_data for item in sublist]
print(len(flat_list))
succeeded = 0
failed = 0
ff = []
SE_good = []
for item in flat_list:
    if 'NotOK' in item:
        failed +=  1
        ff.append(item)
    else:
        succeeded += 1
        if 'XSE'in item:
            SE_good.append(item)

print(succeeded)
print(failed)

18103
18103
0


In [34]:
import datetime
from datetime import timedelta
tstart = datetime.datetime(2018, 6, 23)
strlist = []
for i in range(30):
    dt = timedelta(days=i)
    datetm = (tstart + dt).strftime('%y%m%d')
    strlist.append(datetm)


for datestris in  strlist:
    nbad = 0
    for itis in ff:
        if datestris in itis:
            nbad += 1
    print(datestris, ': ', nbad )
    
    

180623 :  0
180624 :  0
180625 :  0
180626 :  0
180627 :  0
180628 :  0
180629 :  0
180630 :  0
180701 :  0
180702 :  0
180703 :  0
180704 :  0
180705 :  0
180706 :  0
180707 :  0
180708 :  0
180709 :  0
180710 :  0
180711 :  0
180712 :  0
180713 :  0
180714 :  0
180715 :  0
180716 :  0
180717 :  0
180718 :  0
180719 :  0
180720 :  0
180721 :  0
180722 :  0
